# Export project data from Prod to QA for pipeline testing

Production PID's: 100, 278

QA PID's: Copies of above PID's: 75, 74

In [1]:
import redcap

In [2]:
#API URL and API keys import
%run -i -n "credentials.py"

print(dev_url)
print(prod_url)
print(["dev_pid100", "dev_pid278", "prod_pid100", "prod_pid278"])

https://qa-redcap.doh.wa.gov/api/
https://redcap.doh.wa.gov/api/
['dev_pid100', 'dev_pid278', 'prod_pid100', 'prod_pid278']


In [3]:
#Create Project objects
prod100 = redcap.Project(prod_url, prod_pid100)
prod278 = redcap.Project(prod_url, prod_pid278)

dev100 = redcap.Project(dev_url, dev_pid100)
dev278 = redcap.Project(dev_url, dev_pid278)

## Explore PID100 Dataset, Export partial data to DEV env

In [4]:
prod100_df = prod100.export_records(format_type = "df", raw_or_label = "raw")
p100meta = prod100.export_metadata(format_type = "df")

C:\Users\AXG5303\Miniconda3\envs\transform22\lib\site-packages\redcap\methods\base.py:137: DtypeWarning: Columns (397) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframe = pd.read_csv(buf, **df_kwargs)


In [ ]:
#Convert all float values to int
#floating = prod100_df.dtypes == np.floating
#prod100_df.loc[:, floating] = prod100_df.loc[:, floating].astype("Int64")

In [5]:
#import 500 rows of all fields that are "text" type
filt_txt = p100meta["field_type"] == "text"
fields_txt = set(p100meta[filt_txt].index)
fields_txt.remove("record_id") #remove identifier field as it is the index of the dataset

In [6]:
#verify txt fields are the same length with the intersection of txt fields
print(len(fields_txt))
print(len(fields_txt & set(prod100_df.columns)))

#verify that all txt fields exist in dataset 
common = fields_txt & set(prod100_df.columns)
common == fields_txt

257
257


True

In [7]:
#first 500 rows, and all text fields
df_import_txt = prod100_df.iloc[0:500,:].loc[:, list(fields_txt)].copy()

In [8]:
#Convert all float type columns that are defined as text fields in redcap, into Int64 type
floating = df_import_txt.dtypes == np.floating
df_import_txt.loc[:, floating] = df_import_txt.loc[:, floating].astype("Int64")

In [10]:
dev100.import_records(df_import_txt, import_format = "df")

{'count': 500}

In [11]:
df_import_txt

,inq_fac_city,storage_model5,inq_contact_ext,provider_npi24,loc_thu_am_end,org_cmo_licensure_number,org_no_locations,loc_tue_pm_start,org_phone,loc_ship_fax,...,loc_provider_name_22,loc_backup_phone_ext,loc_provider_title_22,isa_date_execute,loc_provider_name_15,loc_provider_license_23,loc_ship_city,loc_provider_license_1,loc_signature_date,loc_backup_email_2
record_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,<NA>,<NA>,12:00,10815,1,12:30,(206) 682-2661,NaN,...,NaN,<NA>,NaN,2020-12-01,NaN,NaN,Seattle,MD60690656,2020-11-10,Yojana.Ghimire@kindred.com
2,NaN,NaN,<NA>,<NA>,12:00,PA10004142,1,13:00,(509) 659-1200,(509) 659-0632,...,NaN,204,NaN,2020-11-03,NaN,NaN,Ritzville,PA10004142,2020-11-02,NaN
3,NaN,MDF-C8V1,<NA>,1124472352,11:59,MD 36063,9,12:00,(206) 624-1144,NaN,...,Amy Rujaya Lavaveshkul,4256813635,PA-C,2020-11-13,Alvaro Lucioni,MD00038526,Seattle,MD60226658,2020-11-09,VMCommandCenter@virginiamason.org
4,NaN,NaN,<NA>,<NA>,12:00,MD00029771,2,12:00,(425) 688-5059,(425) 688-5087,...,"Thomas, Amidon",5822,MD,2020-12-04,"Astier, Alem",AP60347636,Bellevue,MD60448057,2020-11-09,Thomas.DeBord@overlakehospital.org
5,NaN,NaN,<NA>,<NA>,12:00,OP60679363,1,12:30,(509) 775-3333,NaN,...,NaN,344,NaN,NaN,NaN,NaN,Republic,OP60679363,2020-11-05,NaN
6,NaN,NaN,<NA>,<NA>,12:00,RN 00119383,1,13:00,(360) 394-8468,(360) 697-4076,...,NaN,<NA>,NaN,2020-12-02,NaN,NaN,Suquamish,MD00033520,2020-11-04,NaN
7,NaN,NaN,<NA>,<NA>,11:30,RN00120935,2,12:00,(509) 486-2151,(509) 486-3119,...,NaN,7265,NaN,2020-11-06,NaN,NaN,Tonasket,to covid.vaccine@doh.wa.gov,2020-11-04,NaN
8,NaN,NaN,<NA>,<NA>,12:00,MD60153761,1,12:30,(509) 382-2531,NaN,...,NaN,<NA>,NaN,NaN,NaN,NaN,Dayton,MD60173758,2020-11-09,NaN
9,NaN,NaN,<NA>,1407262496,11:59,MD 31167,5,12:00,(360) 426-1611,(360) 427-3617,...,Carlos Rubio-Reyes,28176,MD,NaN,Jennifer Kummerfeldt,AS9799593,Shelton,MD00025623,2020-11-06,NaN


In [ ]:
#import text fields that were floats (now Ints)
df_floats_only = df_import_txt.loc[:,floating].copy()
dev100.import_records(df_floats_only, import_format = "df")

In [ ]:
df_nonfloats = df_import_txt.loc[:,(~floating)].copy()

In [ ]:
dev100.import_records(df_nonfloats, import_format = "df")

In [ ]:
df_nonfloats

In [ ]:
filt = df_import_txt.dtypes == np.floating
df_import_txt.loc[:,filt]

In [ ]:
prod100_df.dtypes

In [ ]:
prod278_df = prod278.export_records(format_type = "df")

In [ ]:
prod100_df.shape

In [ ]:
prod278_df.shape

In [ ]:
p100_df = prod100_df.iloc[0:50,0:21].copy()


In [ ]:
p278_df = prod278_df.iloc[0:50,0:21].copy()

In [ ]:
p100_df["provider_interest"]

In [ ]:
p100_df["provider_interest"] = pd.to_numeric(p100_df["provider_interest"], downcast="integer")

In [ ]:
csv = p100_df.to_csv()

In [ ]:
p100_json = p100_df.to_json(orient = "index")

In [ ]:
dev100.import_records(csv, import_format = "csv")

In [ ]:
dev100.import_records?

In [ ]:
dev100.import_records?

In [ ]:
test = p100_df.iloc[0:2,0:21]

In [ ]:
test

In [ ]:
test.to_json()

In [ ]:
p100meta = prod100.export_metadata(format_type = "df")

In [ ]:
p100meta["field_type"].value_counts()

In [ ]:
numeric = ["checkbox", "yesno", "dropdown", "radio"]

num_fields = list(p100meta[p100meta["field_type"].isin(numeric)].index)

In [ ]:
num_fields_1 = set(num_fields) - set(['loc_setting', 'loc_pop', 'vm_staff_assigned', 'approved_units', 'type_ord_set'])

In [ ]:
prod100_df[list(num_fields_1)]

In [ ]:
p100meta.sort_index(inplace = True)

In [ ]:
[i for i in p100meta.index]

In [ ]:
[i for i in prod100_df.columns.sort_values()]

In [ ]:
loc_set = ['loc_setting___1',
 'loc_setting___10',
 'loc_setting___11',
 'loc_setting___12',
 'loc_setting___13',
 'loc_setting___14',
 'loc_setting___15',
 'loc_setting___16',
 'loc_setting___17',
 'loc_setting___2',
 'loc_setting___3',
 'loc_setting___4',
 'loc_setting___5',
 'loc_setting___6',
 'loc_setting___7',
 'loc_setting___8',
 'loc_setting___9',]

prod100_df[loc_set].dtypes

In [ ]:
p100meta.sort_index(inplace =True)

In [ ]:
p100meta.index

In [ ]:
[i for i in p100meta.index]

In [ ]:
filt = p100meta["form_name"] == "inquiry_enrollment_survey"
p100meta[filt].shape